In [30]:
import os
import json
from psycopg2 import pool
from psycopg2.extras import RealDictCursor
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the connection string from the environment
DATABASE_URL = os.getenv('DATABASE_URL')

# Create a connection pool
connection_pool = pool.SimpleConnectionPool(1, 10, DATABASE_URL)


In [31]:
def format_json_as_paragraph(data, depth=0):
    
    paragraph = ""
    indent = "  " * depth  # Indentation based on depth

    if isinstance(data, dict):
        for key, value in data.items():
            paragraph += f"{indent}{key.capitalize()}: "

            if isinstance(value, (dict, list)):  
                # Recursive call for nested dictionaries/lists
                paragraph += format_json_as_paragraph(value, depth + 1) + "\n"  
            elif isinstance(value, list) and all(isinstance(item, str) for item in value):
                # Special handling for lists of strings
                paragraph += ", ".join(value) + "\n"
            else:
                paragraph += str(value) + "\n"
    elif isinstance(data, list):
        for index, item in enumerate(data):
            if isinstance(item, (dict, list)):
                paragraph += f"{indent}- Item {index + 1}:\n"
                paragraph += format_json_as_paragraph(item, depth + 1) + "\n"  # Recursive call
            else:
                paragraph += f"{indent}- {item}\n"

    return paragraph.strip()  # Remove trailing newline


In [32]:
# Function to retrieve and process cocodata by ID
def get_cocodata_by_id(id):
    with connection_pool.getconn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("SELECT * FROM \"User_Meta\" WHERE id = %s", (id,))
            result = cur.fetchone()

            if result:
                # Convert to regular dictionary 
                data = dict(result)

                # Drop unwanted keys
                for key in ['id', 'seasonColors']:
                    data.pop(key, None)  

                # Add 'gender' key based on 'genderFemale'
                if 'genderFemale' in data:
                    data['gender'] = 'female' if data['genderFemale'] else 'male'
                    del data['genderFemale'] 
                
                # Add 'cm' to height
                if 'height' in data:
                    data['height'] = str(data['height']) + ' cm'

                data_json_string = json.dumps(data)  # Serialize to JSON string
                return data_json_string
            else:
                return None

In [33]:
# Get the ID to search for
id_to_search = 'e053b5ef-4798-4e8c-a805-9fd5a1fe5f0f'

# Fetch and process the cocodata
cocodata_json_string = get_cocodata_by_id(id_to_search)

if cocodata_json_string:
    print("Cocodata found:")
    
    cocodata = json.loads(cocodata_json_string)  
    formatted_output = format_json_as_paragraph(cocodata)
    print(formatted_output) 
else:
    print("Cocodata not found for the given ID.")

# Close all connections in the pool
connection_pool.closeall()

Cocodata found:
Height: 170 cm
Haircolor: black
Dressingsize: 14 UK
Age: 25
Skintone: #D1A885
Stylingseason: Warm Autumn
Iriscolor: dark brown
Ethnicity: India
Bodytype: rectangle
Gender: male


In [35]:
import requests

def get_latlong_weather_description(lat,long):
    latitude = lat  # Approximate latitude of Bhopal
    longitude = long  # Approximate longitude of Bhopal

    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": "weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max",
        "timezone": "Asia/Kolkata"  # Use the correct timezone for Bhopal (IST)
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        daily = data['daily']
        today = daily['time'][0]

        weather_code = daily['weathercode'][0]
        max_temp = daily['temperature_2m_max'][0]
        min_temp = daily['temperature_2m_min'][0]
        precipitation = daily['precipitation_sum'][0]
        max_wind = daily['windspeed_10m_max'][0]

        description = f"Today, the weather is expected to be "

        if weather_code == 0:
            description += "clear skies with "
        elif 1 <= weather_code <= 3:
            description += "mainly clear skies with "
        elif 45 <= weather_code <= 48:
            description += "foggy with "
        elif 51 <= weather_code <= 67:
            description += "drizzle with "
        elif 71 <= weather_code <= 82:
            description += "snowfall with "
        elif 85 <= weather_code <= 86:
            description += "snow showers with "
        elif 95 <= weather_code <= 99:
            description += "thunderstorms with "

        description += f"a high of {max_temp}°C and a low of {min_temp}°C."
        if precipitation > 0:
            description += f" There is a chance of {precipitation} mm of precipitation."
        if max_wind > 15:
            description += f" Winds may gust up to {max_wind} km/h."
        return description
    
    else:
        return "Error fetching weather data."

# Get the weather description
description = get_bhopal_weather_description()
print(description)


Today in Bhopal, the weather is expected to be mainly clear skies with a high of 41.0°C and a low of 30.2°C.
